# Getting the sky on mars

In [8]:
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, Longitude
from astropy import units as u

def get_julian_date(earth_time):
    # Convert Earth time to Julian Date
    earth_time_astropy = Time(earth_time, format='iso', scale='utc')
    return earth_time_astropy.jd

def get_mars_sidereal_time(julian_date, mars_longitude):
    # Mars' sidereal period in days
    mars_sidereal_period = 1.027491
    
    # Convert Julian Date to Astropy Time object
    time = Time(julian_date, format='jd')
    
    # Calculate the fraction of a day since a known Mars LST epoch (JD)
    jd_since_epoch = time.jd - 2451549.5
    mars_days_since_epoch = jd_since_epoch / mars_sidereal_period
    
    # Calculate LST in hours
    lst_hours = (mars_days_since_epoch * 24) % 24
    
    # Adjust for the observer's longitude (in degrees)
    longitude_factor = mars_longitude / 360.0
    lst_adjusted_hours = (lst_hours + 24 * longitude_factor) % 24
    
    # Convert to Astropy Longitude for compatibility with AltAz
    lst = Longitude(lst_adjusted_hours * u.hourangle)
    
    return lst

def get_mars_sky_coords(julian_date, mars_location_longitude, mars_location_latitude):
    # Calculate Local Sidereal Time (LST) for the given Mars location and time
    lst = get_mars_sidereal_time(julian_date, mars_location_longitude)
    
    # Assuming you have the longitude and latitude for your observation point on Mars
    mars_location = EarthLocation(lat=mars_location_latitude*u.deg, lon=mars_location_longitude*u.deg, height=0*u.m)
    
    # Convert to AltAz frame at the given LST and location on Mars
    altaz = AltAz(obstime=Time(julian_date, format='jd'), location=mars_location)
    
    # Example: Get the sky position of Sirius
    sirius = SkyCoord.from_name('Sirius')
    sirius_altaz = sirius.transform_to(altaz)
    
    print(f"Sirius's position on Mars at LST {lst}: Altitude {sirius_altaz.alt}, Azimuth {sirius_altaz.az}")

# Example usage
earth_time = "2023-04-01 12:00:00"
julian_date = get_julian_date(earth_time)
print(julian_date)
get_mars_sky_coords(julian_date, mars_location_longitude=0, mars_location_latitude=0)

2460036.0


TypeError: Coordinate frame AltAz got unexpected keywords: ['lst']